# GeoIPS Single Source Processing Workflow

A GeoIPS processing workflow (procflow) is a Python script that reads satellite data in a variety of formats, sectors and interpolates the data, applies a product algorithm, and outputs the product in a new format (imagery or dataset). This process is shown in the diagram below.

<img src="../figures/single_source_procflow_diagram.png" title="GeoIPS Output Image Here" />

In this tutorial notebook, we have reduced the procflow for a single data source to its most simplified steps. We will walk through each of these steps to demonstrate the most important components of GeoIPS processing.

## Table of Contents
* [Setup and data ingest](#setup-and-data-ingest)
    * [Establish environment](#establish-environment)
    * [Check for available reader](#check-for-available-reader)
    * [Working with file metadata](#working-with-file-metadata)
    * [Read the full dataset](#read-the-full-dataset)
* [Area definitions and sectoring](#area-definitions-and-sectoring¶)
    * [Retreiving the area definition](#retreiving-the-area-definition)
    * [Sectoring the data](#sectoring-the-data)
* [Apply the algorithm](#apply-the-algorithm)
    * [A quick look at interpolation](#a-quick-look-at-interpolation)
* [Generate outputs](#generate-outputs)
    * [Prepare the filenames](#prepare-the-filenames)
    * [Plot the product](#plot-the-product)
* [Putting it all together (3 code blocks)](#putting-it-all-together-3-code-blocks)

# Setup and data ingest

## Establish environment

GeoIPS requires certain environment variables to be set prior to processing. These environment variables are typically not carried over to a Jupyter notebook process. The code block below calls a module that will establish those environment variables here.

In [ ]:
# Set GEOIPS Paths, if they do not already exist
%run utils/set_env.ipy
set_geoips_env()

## Check for available reader

The very first thing we will decide is which product we'd like to produce imagery for and what files will be our input. Let's start with the 89H-Physical product, which applies a particular colorscale to 89GHz data. We will use AMSR2 test data as input.

The file format of our test data is netCDF. Let's check to see if GeoIPS has any built-in readers for our dataset.

In [ ]:
import os
product_name = '89H-Physical'
fnames = [f'{os.getenv("GEOIPS_BASEDIR")}/test_data/test_data_amsr2/data/20200518.062048.gcom-w1.amsr2.amsr2_nesdismanatigcom.x.x.mbt.x.e202005180759470_c202005180937100.nc']

from geoips.geoips_utils import list_entry_points
print(list(filter(lambda x: 'amsr2' in x, list_entry_points('readers')))) # get reader(s) for ssmis

Great! GeoIPS already has a reader established that will handle AMSR2 netCDF files called amsr2_netcdf. Entry points, as used in the list_entry_points function, are object-based name registrations, meaning identifiers can be used to access Python modules no matter how many different repositories may be installed within GeoIPS. We can import amsr2_netcdf through the entrypoints system using the get_reader function.

In [ ]:
from geoips.stable.reader import get_reader
reader = get_reader('amsr2_netcdf')


A brief description of our reader:

Inputs:

* fnames (list): List of strings, full paths to files
* metadata_only (Optional[bool]):
* DEFAULT False
* return before actually reading data if True

Outputs:

* dict of xarray.Datasets: dict of xarray.Dataset objects with required Variables and Attributes: (See geoips/docs :doc:xarray_standards), dict key can be any descriptive dataset id

Every reader in GeoIPS will take a list of filenames as input and return a dictionary of xarray datasets. If you were to create a new reader for GeoIPS, those would be the required inputs and outputs.

## Working with file metadata

Before we read in the file(s) entirely, it is helpful to only get the metadata for our test data. In this case, the returned dictionary will only contain a metadata dataset.

In [ ]:
xobjs = reader(fnames, metadata_only=True)
xobjs['METADATA']

With the metadata, we have what we need to note the variables required for our product algorithm.

In [ ]:
from geoips.dev.product import get_required_variables, get_product_type
variables = get_required_variables(product_name, xobjs['METADATA'].source_name)
print(variables)

Only one variable is required for the 89H-Physical product - the 89GHz horizontally polarized brightness temperature

## Read the full dataset

We have what we need to read in the full dataset. We use the same reader function stored in the "reader" variable. This time, we ensure metadata_only is marked as False.

In [ ]:
xobjs = reader(fnames, metadata_only=False, chans=variables)
list(xobjs.keys())

Again, the returned object is a dictionary of xarray datasets. Only one of these dictionary keys appears to contain what we want. Let's take a look.

In [ ]:
xobjs['Brightness_Temperature_89_GHz_AH']

# Area definitions and sectoring¶

## Retreiving the area definition

In many cases, we will be passing sector information to the processing workflow to get our product in the bounds, resolution, and projection we want. The sector information is stored in a [yaml file](../sectors/amsr2_example_sector.yaml) which will then build an area definition.

An area definition is a geometry object used in the pyresample library. It contains metadata and coordinate information (like area extent) for a sector in addition to the projected coordinate reference system (either in longitude/latitude or X/Y coordinates).

We only need to pass the path to the sector file and the name of the sector (contained in the sector file) to retrieve the area definition.

In [ ]:
area_def_input_dict = {
    'sector_list': ['amsr2-example'],
    'sectorfiles': ["../sectors/amsr2_example_sector.yaml"]
}
from geoips.interface_modules.procflows.single_source import get_area_defs_from_command_line_args
area_defs = get_area_defs_from_command_line_args(area_def_input_dict, xobjs, variables, filter_time=True)
print(area_defs[0])

area_defs is a list of area definitions because GeoIPS is capable of producing output data and imagery for multiple sectors in a single run. In this example we are only running for one sector so we can work with the first instance in area_defs from now on.

For this case we are using a TC sector file (the imagery will be centered on a tropical cyclone). Typically, GeoIPS will use existing deck files with storm information to build a TC area definition. We choose to use a pre-built sector file here instead to keep things simple. Because we are replicating a TC sector with a static sector, we need to make one minor change to get the sector's time information in the correct format.

In [ ]:
area_def = area_defs[0]
from datetime import datetime
area_def.sector_info['synoptic_time'] = datetime.strptime(area_def.sector_info['synoptic_time'], '%Y%m%d%H%M%S')

Let's confirm that GeoIPS recognizes our sector as a TC sector

In [ ]:
from geoips.sector_utils.utils import is_sector_type
is_sector_type(area_defs[0], 'tc')

Furthermore, for TC sectors, we add additional padding around the area definition to ensure the full tropical cyclone is captured in the image.

In [ ]:
from geoips.interface_modules.procflows.single_source import pad_area_definition
pad_area_def = pad_area_definition(area_def, xobjs['METADATA'].source_name)

## Sectoring the data

The next step is to sector the dataset using our area definition. The sectoring function takes an xarray dataset and an area definition object and returns an xarray dataset containing only data within the extent of our sector. We can confirm this by looking at the before and after extents of our dataset:

In [ ]:
xobj = xobjs[list(xobjs.keys())[0]]
print('Extent before sectoring...')
print('Lat min:', xobj.latitude.min().values)
print('Lon min:', xobj.longitude.min().values)
print('Lat max:', xobj.latitude.max().values)
print('Lon max:', xobj.longitude.max().values)

Applying the sectoring function

In [ ]:
from geoips.xarray_utils.data import sector_xarrays
pad_sect_xarrays = sector_xarrays(xobjs, pad_area_def, varlist=variables,
                                  hours_before_sector_time=6, hours_after_sector_time=9, drop=True)

In [ ]:
xobj = pad_sect_xarrays[list(pad_sect_xarrays.keys())[0]]
print('Extent after sectoring...')
print('Lat min:', xobj.latitude.min().values)
print('Lon min:', xobj.longitude.min().values)
print('Lat max:', xobj.latitude.max().values)
print('Lon max:', xobj.longitude.max().values)

# Apply the algorithm

Before moving forward, we need to determine if all of the variables required by the algorithm (stored in the variables object) are contained in our xarray objects.

In [ ]:
all_vars = []
for key, xobj in pad_sect_xarrays.items():
    all_vars += list(xobj.variables.keys())
print('Are all required variables contained in the xarray objects?')
print(set(variables).issubset(all_vars))

Now we can pass our collection of sectored xarrays to the our algorithm which will produce data for the 89H-Physical product.

In [ ]:
from geoips.interface_modules.procflows.single_source import get_alg_xarray
output_format = 'imagery_annotated'
alg_xarray = get_alg_xarray(pad_sect_xarrays, area_def, product_name, resector=False)
alg_xarray

## A quick look at interpolation

As the [diagram at the beginning of this notebook](#geoips-single-source-processing-workflow) suggests, interpolation is a notable step in most GeoIPS procflows. In the single source procflow, interpolation functionality is wrapped into the "get_alg_xarray" method. Thus, the output of "get_alg_xarray" is interpolated data.

We indicate whether interpolation is required and at what point we apply the interpolation by assigning a product_type to the product. 

In [ ]:
from geoips.dev.product import get_product_type
get_product_type(product_name, pad_sect_xarrays['METADATA'].source_name)

The product type for the 89H-Physical product, interp_alg_cmap, indicates that interpolation is applied prior to running the algorithm. If we wanted, we could rearrange to form a different product type called 'alg_interp_cmap' where the algorithm will be applied before interpolating.

The method of interpolation is defined within the GeoIPS system through a yaml config file [(similar to the yaml used to define our static sector)](#Area-definitions-and-sectoring) for our [89H-Physical product](../../geoips/geoips/yaml_configs/product_params/pmw_89/89H-Physical.yaml)¶. The interpolation method is retrieved with the product_name and the data source name.

In [ ]:
from geoips.dev.interp import get_interp
from geoips.dev.product import get_interp_name
interp_func_name = get_interp_name(product_name, pad_sect_xarrays['METADATA'].source_name)
help(get_interp(interp_func_name))

Gaussian interpolation, through the pyresample library, is the interpolation routine requested in the 89H-Physical product yaml. It will use our [area definition, which requests 1400 rows and 1400 columns](#Retreiving-the-area-definition), to interpolate from the original dimensions. Looking at the alg_xarray object's dimensions above we can see dim0 and dim1 are each 1400 in length, which matches the area definition.

# Generate outputs

Prior to plotting, GeoIPS needs to generate the filenames for each output product. For GeoIPS's standard filenaming routine, filenames will contain information about sector, time, and coverage information. There are a number of different filenaming routines that will add more, or different, information to the filenames. However, the standard "geoips_fname" routine is sufficient for this example. Other filenaming modules can be found inside geoips.interface_modules.filename_formats.

## Prepare the filenames

Before generating the filenames, you may want to adjust the output directory environment variable to your desired location. By default, GeoIPS will output imagery and data to /path/to/geoips/install/geoips_outdirs. The geoips_tutorial repository contains an empty outputs directory one step above this notebook file that we'll use in place of the default.

In [ ]:
os.environ['GEOIPS_OUTDIRS'] = os.path.join(os.getcwd(), '../outputs')
print(f'{os.getenv("GEOIPS_OUTDIRS")}')

In [ ]:
from geoips.interface_modules.procflows.single_source import get_output_filenames
filename_formats = ['geoips_fname']
output_dict = {} # in this dictionary, you can request metadata fields to be added to your filename
output_fnames, metadata_fnames = get_output_filenames(
    filename_formats, output_dict, product_name,
    alg_xarray, area_def
)

In [ ]:
print(output_fnames)

The filename(s) are stored as keys in the output_fnames dictionary. A GeoIPS procflow can output more than one product in one run, however our case only has one output image. Now we can pass the filenames and algorithm data to the plotting routine.

## Plot the product

In [ ]:
from geoips.dev.output_config import get_output_format_kwargs
from geoips.interface_modules.procflows.single_source import plot_data
output_format_kwargs = get_output_format_kwargs(output_dict, xarray_obj=alg_xarray, area_def=pad_area_def)
products = plot_data(
    {'output_format': output_format, 'filename_formats': ['geoips_fname']}, 
    alg_xarray, pad_area_def, product_name, output_format_kwargs)

And that's it! the "plot_data" method doesn't return the figure objects themselves. In order to view the image, you will have to locate it inside your output directory. The cell block below should also display the image inside this notebook.

<img src="../outputs/preprocessed/annotated_imagery/x-x-x/x-x-x/89H-Physical/amsr2/20200518.073510.gcom-w1.amsr2.89H-Physical.amsr2-example.99p73.star.1p0.png" width=250 height=250 title="GeoIPS Output Image Here" />

## Putting it all together (3 code blocks)

GeoIPS procflows contain many conditional elements to work with many combinations of user inputs. However, if we zoom in on producing a single image for a single product, the procflow is reduced to very simple steps in not many lines of code. Hopefully you've seen that from the demonstration in this notebook. Just in case, let's strip away the lines of explanation to highlight these steps for a slightly varied product request.

In [ ]:
# resetting to start with empty environment
%reset

In [ ]:
# Import external libraries
import os
from datetime import datetime

# Set GEOIPS Paths, if they do not already exist
%run utils/set_env.ipy
set_geoips_env()

# Import geoips modules
from geoips.interface_modules.procflows.single_source import get_output_filenames
from geoips.dev.output_config import get_output_format_kwargs
from geoips.interface_modules.procflows.single_source import plot_data
from geoips.interface_modules.procflows.single_source import get_alg_xarray
from geoips.xarray_utils.data import sector_xarrays
from geoips.interface_modules.procflows.single_source import pad_area_definition
from geoips.interface_modules.procflows.single_source import get_area_defs_from_command_line_args
from geoips.stable.reader import get_reader
from geoips.dev.product import get_required_variables

In [ ]:
# declare inputs
product_name = 'color89'
output_format = 'imagery_clean'
fnames = [f'{os.getenv("GEOIPS_BASEDIR")}/test_data/test_data_amsr2/data/20200518.062048.gcom-w1.amsr2.amsr2_nesdismanatigcom.x.x.mbt.x.e202005180759470_c202005180937100.nc']
filename_formats = ['geoips_fname']
output_dict = {} # in this dictionary, you can request metadata fields to be added to your filename
area_def_input_dict = {
    'sector_list': ['amsr2-example'],
    'sectorfiles': ["../sectors/amsr2_example_sector.yaml"]
}

In [ ]:
# READING DATA
reader = get_reader('amsr2_netcdf')
xobjs = reader(fnames, metadata_only=True)
variables = get_required_variables(product_name, xobjs['METADATA'].source_name)
xobjs = reader(fnames, metadata_only=False, chans=variables)
# SECTORING DATA
area_def = get_area_defs_from_command_line_args(area_def_input_dict, xobjs, variables, filter_time=True)[0]
area_def.sector_info['synoptic_time'] = datetime.strptime(area_def.sector_info['synoptic_time'], '%Y%m%d%H%M%S')
pad_area_def = pad_area_definition(area_def, xobjs['METADATA'].source_name)
pad_sect_xarrays = sector_xarrays(
    xobjs, pad_area_def, varlist=variables,
    hours_before_sector_time=6, hours_after_sector_time=9, drop=True)
# APPLY ALGORITHM AND INTERPOLATE
alg_xarray = get_alg_xarray(pad_sect_xarrays, area_def, product_name, resector=False)
# PLOT AND STORE OUTPUT
os.environ['GEOIPS_OUTDIRS'] = os.path.join(os.getcwd(), '../outputs')
output_fnames, metadata_fnames = get_output_filenames(
    filename_formats, output_dict, product_name,
    alg_xarray, area_def
)
output_format_kwargs = get_output_format_kwargs(output_dict, xarray_obj=alg_xarray, area_def=pad_area_def)
products = plot_data(
    {'output_format': output_format, 'filename_formats': ['geoips_fname']}, 
    alg_xarray, pad_area_def, product_name, output_format_kwargs)

<img src="../outputs/preprocessed/annotated_imagery/x-x-x/x-x-x/color89/amsr2/20200518.073510.gcom-w1.amsr2.color89.amsr2-example.99p73.star.1p0.png" width=250 height=250 title="GeoIPS Output Image Here" />